# DCA

In [1]:
import numpy as np
import cplex


k = 30
m = 50
R = np.random.uniform(low = 5,high=10,size=(k,m))
h = 3

E = np.sort(R,axis=0)
E = E[k-3:]
E = np.sum(E,axis=0)

def objective(E,X):
  return np.linalg.norm(E-X.dot(R))

In [2]:
def make_team_vector(indices):
  v = np.zeros(k)
  v[indices] = 1
  return v

### Giải bằng greedy tìm global

In [3]:
# brute force find global optima :
best_team = [0,1,2]
lowest_distance = 2e8
for first in range(k-2):
  for second in range(first,k-1):
    for third in range(second,k):
      team = np.zeros(k)
      team[[first,second,third]] = 1 # vector X_i [0,...1...1...1..0]
      d = objective(E,team) # tính khoảng cách 
      if( d < lowest_distance):
        best_team = [first,second,third]
        lowest_distance = d
        
print('best team is : ',best_team,' with distance = ',lowest_distance)

best team is :  [3, 21, 26]  with distance =  42.25652484280789


### Giải G(x) với solver :

In [4]:
ub = [1.0] *k # bound của  0 <= x <=1 
lb = [0.0] *k 
A = R.dot(R.T)
names = ['x'+str(i) for i in range(k)]
linexp = cplex.SparsePair(ind=names,val=[1.0] *k)
row = [i for i in range(k)]
qmat = [[row,A[i]] for i in range(k)] # Hệ số A để truyền vào cplex 
first_part = E.dot(R.T) # Phần E.dot (R.T)


b = -2 * first_part # không gồm phần tau * xl 

p = cplex.Cplex() # make problem

p.objective.set_sense(p.objective.sense.minimize)

p.variables.add(obj=b,ub=ub,lb=lb,names=names) #  0 <= x_i <=1

p.linear_constraints.add(lin_expr=[linexp], 
                       rhs=[3.0],senses="E") # x0+x1 +... + xk = 3
p.objective.set_quadratic(qmat) # set hệ số cho A

p.solve()

CPXPARAM_Read_DataCheck                          1
Number of nonzeros in lower triangle of Q = 435
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.02 sec. (0.01 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 30
  Integer space required    = 30
  Total non-zeros in factor = 465
  Total FP ops to factor    = 9455
Tried aggregator 1 time.
QP Presolve added 0 rows and 30 columns.
Reduced QP has 31 rows, 60 columns, and 525 nonzeros.
Reduced QP objective Q matrix has 30 nonzeros.
Presolve time = 0.03 sec. (0.07 ticks)
Parallel mode: using up to 4 threads for barrier.
Number of nonzeros in lower triangle of A*A' = 465
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.03 ticks)
Summary statistics for Cholesky factor:
  Threads                   = 4
  Rows in Factor            = 31
  Integer space required    = 31
  Total non-zeros in factor = 496
  Total FP ops to factor    = 10416
 Itn      Pri

In [5]:
team = np.array(p.solution.get_values())
team = np.round(team).nonzero()
print(team)
print(objective(E,make_team_vector(team)))

(array([ 3, 10, 26], dtype=int64),)
42.493925644146394


Solver có khi giải được global, có khi lại ra local

### Giải bằng G(x) - h(x) với solver,  $X_i \in R$

In [6]:
def make_problem(b):
  p = cplex.Cplex() # Update b 
  p.objective.set_sense(p.objective.sense.minimize)
  p.variables.add(obj=b,ub=ub,lb=lb,names=names)
  p.linear_constraints.add(lin_expr=[linexp],
                         rhs=[3.0],senses="E")
  p.objective.set_quadratic(qmat)
  return p

In [7]:
list_tau = [0.1,1,10,20,30,50,100,1000,10000]

In [8]:
mat_result = {}
X_0 = np.zeros(k) 

for tau in list_tau:
  dis = []
  list_teams = []
  b_0 = E.dot(R.T) - tau *(2*X_0 - 1)
  p = make_problem(b_0)  
  p.solve()
  l = 0
  while True:
    result = p.solution.get_values()
    team = np.array(result)
    team = np.round(team)
    list_teams.append(team.nonzero())
    dist = objective(E,team)
    dis.append(dist)
    if l > 1 :
      if(dist > dis[l-1]):
        print(str(l),'#######InCorrect######') # sai vì dist bắt buộc phải giảm 
        break
      else:
        if l > 3:
          if(dist == dis[l-3]):
            break # break khi distance không đổi sau 3 vòng 
    new_b = -2 * first_part - tau * (2*team - 1) # update b
    p = make_problem(new_b)
    p.solve()
    l+=1
  mat_result[tau] = list_teams

CPXPARAM_Read_DataCheck                          1
Number of nonzeros in lower triangle of Q = 435
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.01 sec. (0.01 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 30
  Integer space required    = 30
  Total non-zeros in factor = 465
  Total FP ops to factor    = 9455
Tried aggregator 1 time.
QP Presolve added 0 rows and 30 columns.
Reduced QP has 31 rows, 60 columns, and 525 nonzeros.
Reduced QP objective Q matrix has 30 nonzeros.
Presolve time = 0.03 sec. (0.07 ticks)
Parallel mode: using up to 4 threads for barrier.
Number of nonzeros in lower triangle of A*A' = 465
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.03 ticks)
Summary statistics for Cholesky factor:
  Threads                   = 4
  Rows in Factor            = 31
  Integer space required    = 31
  Total non-zeros in factor = 496
  Total FP ops to factor    = 10416
 Itn      Pri

   0  -6.3362855e+05  -1.3041282e+06  1.42e+02  3.00e+01  1.07e+04
   1  -5.2152482e+04  -1.1680594e+05  2.00e-14  0.00e+00  2.15e-10
   2  -5.2183561e+04  -5.4639758e+04  1.03e-14  1.11e-16  1.98e-10
   3  -5.3142858e+04  -5.4939838e+04  3.48e-15  6.66e-16  1.18e-10
   4  -5.3473392e+04  -5.4030484e+04  3.60e-15  1.28e-15  1.47e-10
   5  -5.3668553e+04  -5.3701697e+04  3.12e-15  4.16e-16  1.08e-10
   6  -5.3683020e+04  -5.3683696e+04  2.45e-15  6.17e-16  1.34e-10
   7  -5.3683295e+04  -5.3683307e+04  2.12e-15  4.72e-16  1.32e-10
   8  -5.3683300e+04  -5.3683301e+04  1.72e-15  7.49e-16  1.45e-10
Barrier time = 0.08 sec. (0.37 ticks)

Total time on 4 threads = 0.08 sec. (0.37 ticks)
CPXPARAM_Read_DataCheck                          1
Number of nonzeros in lower triangle of Q = 435
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.01 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 30
  Integer space required    = 30
  Total

Tried aggregator 1 time.
QP Presolve added 0 rows and 30 columns.
Reduced QP has 31 rows, 60 columns, and 525 nonzeros.
Reduced QP objective Q matrix has 30 nonzeros.
Presolve time = 0.00 sec. (0.07 ticks)
Parallel mode: using up to 4 threads for barrier.
Number of nonzeros in lower triangle of A*A' = 465
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.03 ticks)
Summary statistics for Cholesky factor:
  Threads                   = 4
  Rows in Factor            = 31
  Integer space required    = 31
  Total non-zeros in factor = 496
  Total FP ops to factor    = 10416
 Itn      Primal Obj        Dual Obj  Prim Inf Upper Inf  Dual Inf          
   0  -6.3360695e+05  -1.3040850e+06  1.42e+02  3.00e+01  1.07e+04
   1  -5.2150440e+04  -1.1680172e+05  1.86e-14  0.00e+00  2.15e-10
   2  -5.2181595e+04  -5.4643337e+04  2.35e-15  3.33e-16  2.19e-10
   3  -5.3142011e+04  -5.4946388e+04  2.48e-15  7.77e-16  1.28e-10
   4  -5.3474137e+04  -5.4036261e+04  2

   4  -5.3483037e+04  -5.4092042e+04  4.64e-15  9.71e-16  1.41e-10
   5  -5.3690257e+04  -5.3737855e+04  2.03e-15  6.94e-16  1.34e-10
   6  -5.3710668e+04  -5.3711939e+04  1.30e-15  6.40e-16  1.78e-10
   7  -5.3711165e+04  -5.3711212e+04  2.12e-15  7.76e-16  1.44e-10
   8  -5.3711175e+04  -5.3711176e+04  1.91e-15  6.42e-16  1.54e-10
   9  -5.3711176e+04  -5.3711176e+04  1.22e-15  8.89e-16  1.04e-10
Barrier time = 0.11 sec. (0.40 ticks)

Total time on 4 threads = 0.11 sec. (0.40 ticks)
CPXPARAM_Read_DataCheck                          1
Number of nonzeros in lower triangle of Q = 435
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.01 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 30
  Integer space required    = 30
  Total non-zeros in factor = 465
  Total FP ops to factor    = 9455
Tried aggregator 1 time.
QP Presolve added 0 rows and 30 columns.
Reduced QP has 31 rows, 60 columns, and 525 nonzeros.
Reduced QP objectiv

Reduced QP has 31 rows, 60 columns, and 525 nonzeros.
Reduced QP objective Q matrix has 30 nonzeros.
Presolve time = 0.02 sec. (0.07 ticks)
Parallel mode: using up to 4 threads for barrier.
Number of nonzeros in lower triangle of A*A' = 465
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.01 sec. (0.03 ticks)
Summary statistics for Cholesky factor:
  Threads                   = 4
  Rows in Factor            = 31
  Integer space required    = 31
  Total non-zeros in factor = 496
  Total FP ops to factor    = 10416
 Itn      Primal Obj        Dual Obj  Prim Inf Upper Inf  Dual Inf          
   0  -6.3315095e+05  -1.3031730e+06  1.42e+02  3.00e+01  1.07e+04
   1  -5.2107408e+04  -1.1671279e+05  1.05e-14  0.00e+00  2.04e-10
   2  -5.2140240e+04  -5.4718789e+04  2.98e-15  0.00e+00  1.44e-10
   3  -5.3129540e+04  -5.5083719e+04  2.13e-15  5.55e-16  1.25e-10
   4  -5.3495578e+04  -5.4150324e+04  2.09e-15  6.11e-16  1.28e-10
   5  -5.3718325e+04  -5.3766274e+04  

   4  -5.3510333e+04  -5.4205415e+04  1.80e-15  9.16e-16  1.53e-10
   5  -5.3749305e+04  -5.3791206e+04  2.64e-15  7.22e-16  1.25e-10
   6  -5.3770251e+04  -5.3771280e+04  1.81e-15  3.58e-16  1.31e-10
   7  -5.3770884e+04  -5.3770903e+04  1.36e-15  6.68e-16  1.54e-10
   8  -5.3770894e+04  -5.3770895e+04  2.40e-15  7.54e-16  1.09e-10
Barrier time = 0.06 sec. (0.37 ticks)

Total time on 4 threads = 0.06 sec. (0.37 ticks)
CPXPARAM_Read_DataCheck                          1
Number of nonzeros in lower triangle of Q = 435
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.01 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 30
  Integer space required    = 30
  Total non-zeros in factor = 465
  Total FP ops to factor    = 9455
Tried aggregator 1 time.
QP Presolve added 0 rows and 30 columns.
Reduced QP has 31 rows, 60 columns, and 525 nonzeros.
Reduced QP objective Q matrix has 30 nonzeros.
Presolve time = 0.01 sec. (0.07 ticks)


Number of nonzeros in lower triangle of A*A' = 465
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.02 sec. (0.03 ticks)
Summary statistics for Cholesky factor:
  Threads                   = 4
  Rows in Factor            = 31
  Integer space required    = 31
  Total non-zeros in factor = 496
  Total FP ops to factor    = 10416
 Itn      Primal Obj        Dual Obj  Prim Inf Upper Inf  Dual Inf          
   0  -6.3243095e+05  -1.3017330e+06  1.42e+02  3.00e+01  1.07e+04
   1  -5.2039765e+04  -1.1657260e+05  1.29e-14  0.00e+00  2.55e-10
   2  -5.2075481e+04  -5.4837514e+04  2.80e-15  4.44e-16  2.04e-10
   3  -5.3124217e+04  -5.5296157e+04  2.67e-15  9.99e-16  1.60e-10
   4  -5.3544769e+04  -5.4308144e+04  2.40e-15  7.22e-16  1.44e-10
   5  -5.3804362e+04  -5.3856351e+04  2.63e-15  1.42e-15  1.14e-10
   6  -5.3830377e+04  -5.3831350e+04  1.79e-15  1.09e-15  9.40e-11
   7  -5.3830886e+04  -5.3830903e+04  2.99e-15  1.18e-15  1.51e-10
   8  -5.3830894e+04  -5.38

   8  -5.3383028e+04  -5.3383028e+04  2.20e-15  1.15e-15  1.25e-10
Barrier time = 0.06 sec. (0.37 ticks)

Total time on 4 threads = 0.06 sec. (0.37 ticks)
CPXPARAM_Read_DataCheck                          1
Number of nonzeros in lower triangle of Q = 435
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.01 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 30
  Integer space required    = 30
  Total non-zeros in factor = 465
  Total FP ops to factor    = 9455
Tried aggregator 1 time.
QP Presolve added 0 rows and 30 columns.
Reduced QP has 31 rows, 60 columns, and 525 nonzeros.
Reduced QP objective Q matrix has 30 nonzeros.
Presolve time = 0.01 sec. (0.07 ticks)
Parallel mode: using up to 4 threads for barrier.
Number of nonzeros in lower triangle of A*A' = 465
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.02 sec. (0.03 ticks)
Summary statistics for Cholesky factor:
  Threads                 

  Integer space required    = 31
  Total non-zeros in factor = 496
  Total FP ops to factor    = 10416
 Itn      Primal Obj        Dual Obj  Prim Inf Upper Inf  Dual Inf          
   0  -6.0963095e+05  -1.2561330e+06  1.42e+02  3.00e+01  1.07e+04
   1  -4.9849697e+04  -1.1206865e+05  1.48e-14  0.00e+00  2.09e-10
   2  -4.9949481e+04  -5.4740736e+04  2.37e-15  3.33e-16  1.82e-10
   3  -5.2150608e+04  -5.6179204e+04  9.51e-15  5.55e-16  1.73e-10
   4  -5.2681928e+04  -5.4765353e+04  2.80e-15  1.11e-15  1.36e-10
   5  -5.3375989e+04  -5.3693352e+04  1.18e-15  9.85e-16  1.72e-10
   6  -5.3464399e+04  -5.3469569e+04  1.87e-15  9.68e-16  1.78e-10
   7  -5.3465837e+04  -5.3465940e+04  1.98e-15  5.00e-16  1.69e-10
   8  -5.3465884e+04  -5.3465886e+04  1.81e-15  6.44e-16  1.38e-10
   9  -5.3465885e+04  -5.3465885e+04  1.42e-15  6.95e-16  1.54e-10
Barrier time = 0.06 sec. (0.40 ticks)

Total time on 4 threads = 0.06 sec. (0.40 ticks)
CPXPARAM_Read_DataCheck                          1
Number of n

CPXPARAM_Read_DataCheck                          1
Number of nonzeros in lower triangle of Q = 435
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.01 sec. (0.01 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 30
  Integer space required    = 30
  Total non-zeros in factor = 465
  Total FP ops to factor    = 9455
Tried aggregator 1 time.
QP Presolve added 0 rows and 30 columns.
Reduced QP has 31 rows, 60 columns, and 525 nonzeros.
Reduced QP objective Q matrix has 30 nonzeros.
Presolve time = 0.03 sec. (0.07 ticks)
Parallel mode: using up to 4 threads for barrier.
Number of nonzeros in lower triangle of A*A' = 465
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.03 ticks)
Summary statistics for Cholesky factor:
  Threads                   = 4
  Rows in Factor            = 31
  Integer space required    = 31
  Total non-zeros in factor = 496
  Total FP ops to factor    = 10416
 Itn      Pri

In [9]:
mat_result

{0.1: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],
 1: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],
 10: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],
 20: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],
 30: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],


In [10]:
objective(E,make_team_vector([13,18,26]))

46.66714757002837

In [11]:
objective(E,make_team_vector([12,15,23]))

50.90685274829679

### Giải G(x) - H(x), điều kiện $X_i \in N$

In [12]:
def make_problem2(b):
  p = cplex.Cplex() # Update b 
  p.objective.set_sense(p.objective.sense.minimize)
  p.variables.add(obj=b,ub=ub,lb=lb,names=names,types=['I']*k) # int
  p.linear_constraints.add(lin_expr=[linexp],
                         rhs=[3.0],senses="E")
  p.objective.set_quadratic(qmat)
  return p

In [13]:
mat_result = {}
X_0 = np.zeros(k) 

for tau in list_tau:
  dis = []
  list_teams = []
  b_0 = E.dot(R.T) - tau *(2*X_0 - 1)
  p = make_problem2(b_0)  
  p.solve()
  l = 0
  while True:
    result = p.solution.get_values()
    team = np.array(result)
    team = np.round(team)
    list_teams.append(team.nonzero())
    dist = objective(E,team)
    dis.append(dist)
    if l > 1 :
      if(dist > dis[l-1]):
        print(str(l),'#######InCorrect######') # sai vì dist bắt buộc phải giảm 
        break
      else:
        if l > 3:
          if(dist == dis[l-3]):
            break
    new_b = -2 * first_part - tau * (2*team - 1) # update b
    p = make_problem2(new_b)
    p.solve()
    l+=1
  mat_result[tau] = list_teams

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve added 870 rows and 435 columns.
Reduced MIP has 871 rows, 465 columns, and 1770 nonzeros.
Reduced MIP has 465 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.12 ticks)
Found incumbent of value 43756.171083 after 0.02 sec. (0.41 ticks)
Probing time = 0.00 sec. (0.12 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 435 rows and 0 columns.
Reduced MIP has 436 rows, 465 columns, and 1335 nonzeros.
Reduced MIP has 465 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.06 ticks)
Probing time = 0.02 sec. (0.09 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.46 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap


      0     0   -61329.1778    11   -53497.4139  ZeroHalf: 12       61   14.64%
      0     0   -61296.2199    13   -53497.4139  ZeroHalf: 19       68   14.58%
      0     0   -61263.9471    11   -53497.4139  ZeroHalf: 14       77   14.52%
      0     0   -61255.7657    13   -53497.4139   ZeroHalf: 7       81   14.50%
      0     0   -61215.1883     6   -53497.4139   ZeroHalf: 6       87   14.43%
      0     0   -61196.7603     6   -53497.4139  ZeroHalf: 12       90   14.39%
      0     0   -61192.3321     6   -53497.4139   ZeroHalf: 8       92   14.38%
      0     0   -61191.4699     8   -53497.4139  ZeroHalf: 10       95   14.38%
*     0+    0                       -53681.1946   -61191.4699            13.99%
      0     2   -61191.4699     8   -53681.1946   -60872.3689       95   13.40%
Elapsed time = 0.25 sec. (25.25 ticks, tree = 0.01 MB, solutions = 3)

Zero-half cuts applied:  24

Root node processing (before b&c):
  Real time             =    0.20 sec. (25.04 ticks)
Parallel b&c

   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                        43758.8711        0.0000           100.00%
      0     0    35296.8620     6    43758.8711    35296.8620        5   19.34%
*     0+    0                        42983.3289    35296.8620            17.88%
      0     0    35454.7832     6    42983.3289       Cuts: 9       11   17.52%
      0     0    35576.0955     8    42983.3289       Cuts: 7       19   17.23%
      0     0    35617.6694     6    42983.3289  ZeroHalf: 16       26   17.14%
      0     0    35626.2590     9    42983.3289   ZeroHalf: 7       28   17.12%
      0     0    35663.3015    10    42983.3289  ZeroHalf: 17       36   17.03%
      0     0    35719.9006     9    42983.3289   ZeroHalf: 9       40   16.90%
      0     0    35757.7692     8    42983.3289  ZeroHalf: 11       46   16.81%
      0     0    35766.6240    11    42983.3289  ZeroHalf: 12       52   16.79%
      0     0    35775.7694     9    429

  Real time             =    0.11 sec. (25.85 ticks)
  Sync time (average)   =    0.06 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.27 sec. (50.81 ticks)
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve added 870 rows and 435 columns.
Reduced MIP has 871 rows, 465 columns, and 1770 nonzeros.
Reduced MIP has 465 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.12 ticks)
Found incumbent of value -51188.837527 after 0.00 sec. (0.41 ticks)
Probing time = 0.00 sec. (0.12 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 435 rows and 0 columns.
Reduced MIP has 436 rows, 465 columns, and 1335 nonzeros.
Reduced MIP has 465 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (1.05 ticks)
Probing time = 0.00 sec. (0.09 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: determini

      0     0    35802.7694     9    43010.3289   ZeroHalf: 8       56   16.76%
      0     0    35823.8226    10    43010.3289   ZeroHalf: 6       64   16.71%
      0     0    35843.4450     6    43010.3289  ZeroHalf: 11       72   16.66%
      0     0    35865.3924    12    43010.3289  ZeroHalf: 12       76   16.61%
      0     0    35899.9419     8    43010.3289   ZeroHalf: 8       81   16.53%
      0     0    35907.9155     8    43010.3289  ZeroHalf: 13       88   16.51%
      0     0    35911.1836     9    43010.3289  ZeroHalf: 10       91   16.51%
      0     0    35941.7033    11    43010.3289  ZeroHalf: 16       99   16.43%
      0     0    35958.7211    10    43010.3289  ZeroHalf: 10      106   16.40%
      0     0    35961.4627    10    43010.3289   ZeroHalf: 6      108   16.39%
*     0+    0                        42774.6553    35961.4627            15.93%
      0     2    35961.4627    10    42774.6553    36748.5773      108   14.09%
Elapsed time = 0.20 sec. (29.74 ticks, t

...

      0     0    36211.7033    11    43280.3289  ZeroHalf: 16       99   16.33%
      0     0    36228.7211    10    43280.3289  ZeroHalf: 10      106   16.29%
      0     0    36231.4627    10    43280.3289   ZeroHalf: 6      108   16.29%
*     0+    0                        43044.6553    36231.4627            15.83%
      0     2    36231.4627    10    43044.6553    37018.5773      108   14.00%
Elapsed time = 0.22 sec. (29.74 ticks, tree = 0.01 MB, solutions = 3)

Zero-half cuts applied:  28

Root node processing (before b&c):
  Real time             =    0.20 sec. (29.55 ticks)
Parallel b&c, 4 threads:
  Real time             =    0.11 sec. (30.28 ticks)
  Sync time (average)   =    0.05 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.31 sec. (59.82 ticks)
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve added 870 rows and 435 columns.
Reduced MIP has 871 rows, 465 columns, and 

      0     0   -61432.6602     9   -53597.3139  ZeroHalf: 14       25   14.62%
      0     0   -61418.5019     8   -53597.3139  ZeroHalf: 10       27   14.59%
      0     0   -61323.5640     8   -53597.3139  ZeroHalf: 11       35   14.42%
      0     0   -61293.9553     6   -53597.3139  ZeroHalf: 11       38   14.36%
      0     0   -61287.7368     8   -53597.3139   ZeroHalf: 6       40   14.35%
      0     0   -61283.9538     6   -53597.3139  ZeroHalf: 11       42   14.34%
      0     0   -61275.3485     9   -53597.3139   ZeroHalf: 9       48   14.33%
      0     0   -61211.3482     6   -53597.3139  ZeroHalf: 17       59   14.21%
      0     0   -61195.4867    13   -53597.3139  ZeroHalf: 11       62   14.18%
      0     0   -61154.8623     6   -53597.3139   ZeroHalf: 8       71   14.10%
      0     0   -61150.0174     9   -53597.3139   ZeroHalf: 9       72   14.09%
*     0+    0                       -53980.8946   -61150.0174            13.28%
      0     2   -61150.0174     9   -539

Presolve time = 0.00 sec. (1.05 ticks)
Probing time = 0.00 sec. (0.09 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.46 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                       -48191.8375  -578795.2125              --- 
      0     0   -60428.9845     6   -48191.8375   -60428.9845        5   25.39%
*     0+    0                       -52597.9998   -60428.9845            14.89%
      0     0   -60136.2509     6   -52597.9998       Cuts: 6        9   14.33%
      0     0   -59970.9589     6   -52597.9998       Cuts: 8       14   14.02%
      0     0   -59946.9840     8   -52597.9998   ZeroHalf: 6       17   13.97%
      0     0   -59770.8053     8   -52597.9998  ZeroHalf: 14       25   13.64%
      0     0   -59623.1048  

      0     0   -59623.1048     8   -52597.9998  ZeroHalf: 11       31   13.36%
      0     0   -59562.3323     6   -52597.9998  ZeroHalf: 12       35   13.24%
      0     0   -59535.1029     9   -52597.9998   ZeroHalf: 7       38   13.19%
      0     0   -59486.0738     6   -52597.9998   ZeroHalf: 8       42   13.10%
      0     0   -59470.7951     9   -52597.9998   ZeroHalf: 6       44   13.07%
      0     0   -59362.6604    11   -52597.9998  ZeroHalf: 15       51   12.86%
      0     0   -59170.1219    10   -52597.9998  ZeroHalf: 10       63   12.50%
      0     0   -59068.7067     9   -52597.9998   ZeroHalf: 8       70   12.30%
      0     0   -59044.7919    10   -52597.9998  ZeroHalf: 11       77   12.26%
      0     0   -59023.7107     9   -52597.9998  ZeroHalf: 11       80   12.22%
      0     0   -58976.8301    11   -52597.9998  ZeroHalf: 14       85   12.13%
      0     0   -58935.4500     8   -52597.9998   ZeroHalf: 6       92   12.05%
      0     0   -58911.5824    13   -525

      0     2    65881.1836     9    72744.6553    66718.5773       91    8.28%
Elapsed time = 0.19 sec. (24.21 ticks, tree = 0.01 MB, solutions = 3)

Zero-half cuts applied:  24
Lift and project cuts applied:  1

Root node processing (before b&c):
  Real time             =    0.17 sec. (24.02 ticks)
Parallel b&c, 4 threads:
  Real time             =    0.11 sec. (28.83 ticks)
  Sync time (average)   =    0.06 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.28 sec. (52.85 ticks)
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve added 870 rows and 435 columns.
Reduced MIP has 871 rows, 465 columns, and 1770 nonzeros.
Reduced MIP has 465 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.12 ticks)
Found incumbent of value -21191.837527 after 0.01 sec. (0.41 ticks)
Probing time = 0.00 sec. (0.12 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 435 rows 

In [14]:
mat_result

{0.1: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],
 1: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],
 10: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],
 20: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],
 30: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],


## Nhận xét : 
- Solver giải G(x) và DCA G(x)-h(x) đều ra local optima, chưa phải global
- Khi $\tau$ lớn, G(X) không có ý nghĩa 